In [13]:
import warnings
warnings.filterwarnings("ignore")

from data_loading import *

from xai_agg.agg_exp import *
from xai_agg.utils import *

import pandas as pd
import numpy as np

In [14]:
dataset_name, preprocessed_data, categorical_features, X, y, X_train, X_test, y_train, y_test, clf = load_taiwan_data()

Accuracy: 0.82
ROC AUC: 0.6614906832298137


In [15]:
evaluator = ExplanationModelEvaluator(clf, X_train, categorical_features)
evaluator.init()

Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1737 - val_loss: 1.1493
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - loss: 1.2335 - val_loss: 1.0854
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - loss: 1.1586 - val_loss: 1.0113
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - loss: 0.9816 - val_loss: 0.9418
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - loss: 1.0594 - val_loss: 0.8875
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - loss: 0.9327 - val_loss: 0.8469
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step - loss: 0.8947 - val_loss: 0.8177
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 530us/step - loss: 0.8415 - val_loss: 0.7975
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - loss: 0.7605 - val_loss: 0.7832
Epoch 10/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step - loss: 0.9205 - val_loss: 0.7724
Epoch 11/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - loss: 0.8133 - val_loss: 0.7637
Epoch 12/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 

# Generating metric vectors and calculating correlation between them

In [16]:
# Getting a few reanknings
exps = [LimeWrapper(clf, X_train, categorical_features), ShapTabularTreeWrapper(clf, X_train, categorical_features), AnchorWrapper(clf, X_train, categorical_features)]

indexes = np.random.choice(X_test.index, 100, replace=False)
comp_arr = [] # original, new1, new2, ...

In [17]:
import concurrent.futures

def process_index(idx):
    local_comp_arr = []
    for exp in exps:
        row = []
        explanation = exp.explain_instance(X_test.loc[idx])
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], g_x=explanation, ranked=False))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], g_x=explanation, ranked=False))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], g_x=explanation, ranked=True, rank_alg="sum"))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], g_x=explanation, ranked=True, rank_alg="percentile"))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], g_x=explanation, ranked=True, rank_alg="avg"))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], g_x=explanation, ranked=True, rank_alg="inverse"))
        
        # row.append(evaluator.complexity(exp, X_test.loc[idx], explanation=explanation))
        # row.append(evaluator.nrc(exp, X_test.loc[idx], explanation=explanation))
        local_comp_arr.append(row)
    return local_comp_arr
    
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(process_index, indexes))

for result in results:
    comp_arr.extend(result)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now ret

In [18]:
from scipy.stats import spearmanr

comp_arr = np.array(comp_arr)
pearson_corr = [np.corrcoef(comp_arr[:, 0], comp_arr[:, i])[0, 1] for i in range(1, comp_arr.shape[1])]
spearman_corr = [spearmanr(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]

print(dataset_name)
print("Pearson Correlation:", pearson_corr)
print("Spearman Correlation:", spearman_corr)

TAIWAN
Pearson Correlation: [0.8479440293240804, 0.3517271522887647, 0.34852359409445755, nan, 0.3053430657483569]
Spearman Correlation: [0.771681463127368, 0.17514905721174678, 0.16883787597639974, nan, 0.1091932132579251]
